https://github.com/opendatalab/MinerU/blob/master/docs/README_Windows_CUDA_Acceleration_zh_CN.md

先不采用docker
本地环境：    pip install -U magic-pdf[full] -i https://mirrors.aliyun.com/pypi/simple
model 已下载，位置可见用户目录中的magic.json
win中安装了cuda12.6 按上述测试方案重装了torch

4-16 发现不支持过长的文件名，可修改windows注册表

用rag环境

用多个GPU要在必须在导入任何使用 GPU 的库（如 torch）之前设置。因为 PyTorch 在导入时会初始化 CUDA 设备，如果在导入后设置环境变量，将不会生效。

测试一下加速办法
：多进程
多线程
OCRGPU pip install paddlepaddle-gpu==2.6.1


In [6]:
import os
os.getcwd()
print(os.getcwd())

d:\DataAnalysis\arxiv


In [3]:
!cd

d:\DataAnalysis\arxiv


In [1]:
#work version
# Copyright (c) Opendatalab. All rights reserved.
import os

# 设置使用的GPU（如使用第1块GPU，索引为1）
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from magic_pdf.data.data_reader_writer import FileBasedDataWriter, FileBasedDataReader
from magic_pdf.data.dataset import PymuDocDataset
from magic_pdf.model.doc_analyze_by_custom_model import doc_analyze
from magic_pdf.config.enums import SupportedPdfParseMethod

# args
#__dir__ = os.path.dirname(os.path.abspath(__file__))
__dir__ = os.getcwd()


# Path to the parsed list file
parsed_list_file = os.path.join(__dir__, "parsed_list.txt")

# Load the parsed list if it exists
if os.path.exists(parsed_list_file):
    with open(parsed_list_file, "r") as f:
        parsed_list = set(f.read().splitlines())
else:
    with open(parsed_list_file, "w") as f:
        f.write("")  # Create an empty file
    parsed_list = set()

# Get all files in the downloads directory
downloads_dir = os.path.join(__dir__, "downloads")
#downloads_dir = os.path.join(__dir__, "PDF")
all_files = [os.path.join(downloads_dir, f) for f in os.listdir(downloads_dir) if f.endswith(".pdf")]

# Process files not in the parsed list
for pdf_file_name in all_files[:]:
    if pdf_file_name not in parsed_list:
        try:
            #name_without_extension = '.'.join(os.path.basename(pdf_file_name).split('.')[0:-1]) #不能支持那么长
            name_without_extension = '.'.join(os.path.basename(pdf_file_name).split('.')[0:2]) #不能支持那么长

            print(name_without_extension)

            # Prepare environment
            local_image_dir = os.path.join(__dir__, "output", name_without_extension, "images")
            local_md_dir = os.path.join(__dir__, "output", name_without_extension)
            image_dir = str(os.path.basename(local_image_dir))
            os.makedirs(local_image_dir, exist_ok=True)

            image_writer, md_writer = FileBasedDataWriter(local_image_dir), FileBasedDataWriter(local_md_dir)

            # Read bytes
            reader1 = FileBasedDataReader("")

            pdf_bytes = reader1.read(pdf_file_name)

            # Create Dataset Instance
            ds = PymuDocDataset(pdf_bytes)

            # Inference
            if ds.classify() == SupportedPdfParseMethod.OCR:
                infer_result = ds.apply(doc_analyze, ocr=True)
                pipe_result = infer_result.pipe_ocr_mode(image_writer)
            else:
                infer_result = ds.apply(doc_analyze, ocr=False)
                pipe_result = infer_result.pipe_txt_mode(image_writer)

            # Draw layout and spans results
            #model_inference_result = infer_result.get_infer_res()
            #md_content = pipe_result.get_markdown(image_dir)
            #middle_json_content = pipe_result.get_middle_json()
            #content_list_content = pipe_result.get_content_list(image_dir)


            pipe_result.draw_layout(os.path.join(local_md_dir, f"{name_without_extension}_layout.pdf"))
            pipe_result.draw_span(os.path.join(local_md_dir, f"{name_without_extension}_spans.pdf"))
            pipe_result.dump_md(md_writer, f"{name_without_extension}.md", image_dir)
            pipe_result.dump_content_list(md_writer, f"{name_without_extension}_content_list.json", image_dir)
            pipe_result.dump_middle_json(md_writer, f'{name_without_extension}_middle.json')

            # Add the file to the parsed list
            parsed_list.add(pdf_file_name)

        except Exception as e:
            print(f"Error processing file {pdf_file_name}: {e}")

# Save the updated parsed list
with open(parsed_list_file, "w") as f:
    f.write("\n".join(parsed_list))


d:\Tools\anaconda3\envs\rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-25 19:07:57.119 | INFO     | magic_pdf.data.dataset:__init__:157 - lang: None


2504.08386v1


2025-05-25 19:07:57.598 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 5, text_len: 24328, cid_chars_radio: 0.00020582060675914873
2025-05-25 19:08:01.346 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:may_batch_image_analyze:275 - gpu_memory: 24 GB, batch_ratio: 16
2025-05-25 19:08:01.352 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:68 - DocAnalysis init, this may take some times, layout_model: doclayout_yolo, apply_formula: True, apply_ocr: True, apply_table: True, table_model: rapid_table, lang: None
2025-05-25 19:08:01.352 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:82 - using device: cuda
2025-05-25 19:08:01.353 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:86 - using models_dir: C:\Users\zhuzhanwen\.cache\huggingface\hub\models--opendatalab--PDF-Extract-Kit-1.0\snapshots\95817b4b2321769155f05c8d7e2f5a6b6da9e662/models
2025-05-25 19:08:12.211 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:164 - DocAnalysis ini

2504.08490v1


2025-05-25 19:08:24.386 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 21585, cid_chars_radio: 0.0
2025-05-25 19:08:24.481 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:may_batch_image_analyze:275 - gpu_memory: 24 GB, batch_ratio: 16
OCR-det Predict: 100%|██████████| 8/8 [00:01<00:00,  6.42it/s]
Table Predict: 0it [00:00, ?it/s]
Processing pages: 100%|██████████| 8/8 [00:00<00:00, 13.86it/s]
2025-05-25 19:08:27.956 | INFO     | magic_pdf.data.dataset:__init__:157 - lang: None


...

2025-05-25 20:32:57.331 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 27602, cid_chars_radio: 0.0
2025-05-25 20:32:57.516 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:may_batch_image_analyze:275 - gpu_memory: 24 GB, batch_ratio: 16
OCR-rec Predict: 100%|██████████| 1/1 [00:00<00:00, 142.88it/s]
2025-05-25 20:33:04.159 | INFO     | magic_pdf.data.dataset:__init__:157 - lang: None


2505.05031v1


2025-05-25 20:33:05.610 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 2, text_len: 34294, cid_chars_radio: 5.833965346245843e-05
2025-05-25 20:33:05.944 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:may_batch_image_analyze:275 - gpu_memory: 24 GB, batch_ratio: 16
Processing pages: 100%|██████████| 21/21 [00:03<00:00,  5.93it/s]
2025-05-25 20:33:22.697 | INFO     | magic_pdf.data.dataset:__init__:157 - lang: None


2505.05057v1


2025-05-25 20:33:23.899 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 1, text_len: 57480, cid_chars_radio: 1.739977728285078e-05
2025-05-25 20:33:24.151 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:may_batch_image_analyze:275 - gpu_memory: 24 GB, batch_ratio: 16
OCR-rec Predict: 100%|██████████| 2/2 [00:00<00:00, 124.92it/s]


In [ ]:
# demo batch


import os

# 设置使用的GPU（如使用第1块GPU，索引为1）
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

            
from pathlib import Path
from magic_pdf.data.batch_build_dataset import batch_build_dataset
from magic_pdf.tools.common import batch_do_parse

import torch
print(f"可用GPU数量: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

# 检查当前可见的GPU
print(f"当前可见的GPU: {os.environ['CUDA_VISIBLE_DEVICES']}")
#torch.cuda.set_device(1)
# 查看当前可用的GPU数量和名称
print(f"可用GPU数量: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")


__dir__ = os.getcwd()
parsed_list_file = os.path.join(__dir__, "parsed_list.txt")

# Load the parsed list if it exists
if os.path.exists(parsed_list_file):
    with open(parsed_list_file, "r") as f:
        parsed_list = set(f.read().splitlines())
else:
    with open(parsed_list_file, "w") as f:
        f.write("")  # Create an empty file
    parsed_list = set()

# Get all files in the downloads directory
downloads_dir = os.path.join(__dir__, "downloads")
#downloads_dir = os.path.join(__dir__, "PDF")
all_files = [os.path.join(downloads_dir, f) for f in os.listdir(downloads_dir) if f.endswith(".pdf")]
all_files_tobe = []
# Process files not in the parsed list
for pdf_file_name in all_files:
    if pdf_file_name not in parsed_list:
        all_files_tobe.append(pdf_file_name)
 

output_dir = os.path.join(__dir__, "output")
method = "auto"
lang = ""



# 构建数据集，4个worker
# 将 all_files_tobe 拆分为每100个一组的子列表
def chunk_list(lst, chunk_size):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

chunks = list(chunk_list(all_files_tobe, 100))

# 示例：打印每个分组的文件数量

for idx, chunk in enumerate(chunks):
    print(f"Chunk {idx+1}: {len(chunk)} files")

    datasets = batch_build_dataset(chunk, 10, lang)

    # 可选：设置每批次推理的页数
    #os.environ["MINERU_MIN_BATCH_INFERENCE_SIZE"] = "200"

    # 批量解析
    output_namelist = ['.'.join(os.path.basename(doc_path).split('.')[0:2]) for doc_path in chunk ]

    batch_do_parse(output_dir, output_namelist, datasets, method)

#datasets = batch_build_dataset(all_files_tobe[:100], 10, lang)
#output_namelist = ['.'.join(os.path.basename(doc_path).split('.')[0:2]) for doc_path in all_files_tobe[:100] ]
#batch_do_parse(output_dir, output_namelist, datasets, method)

可用GPU数量: 1
GPU 0: NVIDIA GeForce RTX 4090
当前可见的GPU: 1
可用GPU数量: 1
GPU 0: NVIDIA GeForce RTX 4090
Chunk 1: 100 files


2025-05-23 17:34:03.823 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 4, text_len: 32734, cid_chars_radio: 0.00012230171833914265
2025-05-23 17:34:04.579 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 38, text_len: 35073, cid_chars_radio: 0.0010917344212371075
2025-05-23 17:34:05.522 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 11, text_len: 43027, cid_chars_radio: 0.0002560879079945989
2025-05-23 17:34:05.901 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 22911, cid_chars_radio: 0.0
2025-05-23 17:34:06.583 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 1, text_len: 41691, cid_chars_radio: 2.3990020151616926e-05
2025-05-23 17:34:07.284 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 3, text_len: 36020, cid_chars_radio: 8.333564821245035e-05
2025-05-23 17:34:08.211 | INFO     | magic_pdf.libs.pdf_check:dete

Chunk 2: 100 files


2025-05-23 17:57:00.147 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 19284, cid_chars_radio: 0.0
2025-05-23 17:57:00.698 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 24180, cid_chars_radio: 0.0
2025-05-23 17:57:01.554 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 2, text_len: 35214, cid_chars_radio: 5.681818181818182e-05
2025-05-23 17:57:02.223 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 42705, cid_chars_radio: 0.0
2025-05-23 17:57:02.963 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 2, text_len: 34275, cid_chars_radio: 5.8375412276349205e-05
2025-05-23 17:57:03.563 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 36733, cid_chars_radio: 0.0
2025-05-23 17:57:04.455 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 36389, cid_

Chunk 3: 100 files
MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: 

...

2025-05-23 22:30:59.270 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 21, text_len: 32500, cid_chars_radio: 0.000649471144924847
2025-05-23 22:31:00.169 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 30410, cid_chars_radio: 0.0
2025-05-23 22:31:01.097 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 27180, cid_chars_radio: 0.0
2025-05-23 22:31:02.433 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 34384, cid_chars_radio: 0.0
2025-05-23 22:31:03.184 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 29336, cid_chars_radio: 0.0
2025-05-23 22:31:04.068 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 27327, cid_chars_radio: 0.0
2025-05-23 22:31:05.344 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 16, text_len: 57310, cid_chars_radio: 0.000

MuPDF error: syntax error: could not parse color space (264 0 R)

MuPDF error: syntax error: could not parse color space (532 0 R)



2025-05-23 22:32:47.615 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 25989, cid_chars_radio: 0.0


MuPDF error: syntax error: could not parse color space (264 0 R)

MuPDF error: syntax error: could not parse color space (532 0 R)



2025-05-23 22:32:48.968 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 24739, cid_chars_radio: 0.0
2025-05-23 22:32:50.917 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 29727, cid_chars_radio: 0.0
2025-05-23 22:32:52.428 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 1, text_len: 41692, cid_chars_radio: 2.3990020151616926e-05
2025-05-23 22:32:53.595 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 1, text_len: 27641, cid_chars_radio: 3.6187305493232976e-05
2025-05-23 22:32:54.462 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 52346, cid_chars_radio: 0.0
2025-05-23 22:32:55.843 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 53083, cid_chars_radio: 0.0
2025-05-23 22:32:56.991 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 28510, cid

In [10]:
all_files_tobe[:10]

['d:\\DataAnalysis\\arxiv\\downloads\\2005.11401v4.Retrieval_Augmented_Generation_for_Knowledge_Intensive_NLP_Tasks.pdf',
 'd:\\DataAnalysis\\arxiv\\downloads\\2006.05405v5.Retrieval_Augmented_Generation_for_Code_Summarization_via_Hybrid_GNN.pdf',
 'd:\\DataAnalysis\\arxiv\\downloads\\2102.04643v1.Efficient_Retrieval_Augmented_Generation_from_Unstructured_Knowledge_for_Task_Oriented_Dialog.pdf',
 'd:\\DataAnalysis\\arxiv\\downloads\\2104.08610v1.Zero_shot_Slot_Filling_with_DPR_and_RAG.pdf',
 'd:\\DataAnalysis\\arxiv\\downloads\\2108.13934v2.Robust_Retrieval_Augmented_Generation_for_Zero_shot_Slot_Filling.pdf',
 'd:\\DataAnalysis\\arxiv\\downloads\\2109.03754v2.Memory_and_Knowledge_Augmented_Language_Models_for_Inferring_Salience_in_Long_Form_Stories.pdf',
 'd:\\DataAnalysis\\arxiv\\downloads\\2109.07263v2.End_to_End_Learning_of_Flowchart_Grounded_Task_Oriented_Dialogs.pdf',
 'd:\\DataAnalysis\\arxiv\\downloads\\2112.08688v2.Evidentiality_guided_Generation_for_Knowledge_Intensive_NLP_Ta

In [14]:
len(all_files_tobe)

78

In [12]:
chunk

['d:\\DataAnalysis\\arxiv\\downloads\\2005.11401v4.Retrieval_Augmented_Generation_for_Knowledge_Intensive_NLP_Tasks.pdf',
 'd:\\DataAnalysis\\arxiv\\downloads\\2006.05405v5.Retrieval_Augmented_Generation_for_Code_Summarization_via_Hybrid_GNN.pdf',
 'd:\\DataAnalysis\\arxiv\\downloads\\2102.04643v1.Efficient_Retrieval_Augmented_Generation_from_Unstructured_Knowledge_for_Task_Oriented_Dialog.pdf',
 'd:\\DataAnalysis\\arxiv\\downloads\\2104.08610v1.Zero_shot_Slot_Filling_with_DPR_and_RAG.pdf',
 'd:\\DataAnalysis\\arxiv\\downloads\\2108.13934v2.Robust_Retrieval_Augmented_Generation_for_Zero_shot_Slot_Filling.pdf',
 'd:\\DataAnalysis\\arxiv\\downloads\\2109.03754v2.Memory_and_Knowledge_Augmented_Language_Models_for_Inferring_Salience_in_Long_Form_Stories.pdf',
 'd:\\DataAnalysis\\arxiv\\downloads\\2109.07263v2.End_to_End_Learning_of_Flowchart_Grounded_Task_Oriented_Dialogs.pdf',
 'd:\\DataAnalysis\\arxiv\\downloads\\2112.08688v2.Evidentiality_guided_Generation_for_Knowledge_Intensive_NLP_Ta

In [ ]:
from concurrent.futures import ProcessPoolExecutor
import os

def process_pdf(pdf_file_name):
    # 你的单文件处理逻辑，和原来 for 循环体一样
    # ...
    return pdf_file_name

if __name__ == "__main__":
    downloads_dir = os.path.join(os.getcwd(), "downloads")
    all_files = [os.path.join(downloads_dir, f) for f in os.listdir(downloads_dir) if f.endswith(".pdf")]

    with ProcessPoolExecutor(max_workers=4) as executor:  # 4个进程，可根据CPU/GPU数量调整
        for result in executor.map(process_pdf, all_files):
            print(f"Processed: {result}")

可用GPU数量: 2
GPU 0: NVIDIA GeForce RTX 4090
GPU 1: NVIDIA GeForce RTX 4090
当前可见的GPU: 0,1


In [4]:
#for test
#Copyright (c) Opendatalab. All rights reserved.
import os

from magic_pdf.data.data_reader_writer import FileBasedDataWriter, FileBasedDataReader
from magic_pdf.data.dataset import PymuDocDataset
from magic_pdf.model.doc_analyze_by_custom_model import doc_analyze
from magic_pdf.config.enums import SupportedPdfParseMethod

# args
#__dir__ = os.path.dirname(os.path.abspath(__file__))
__dir__ = os.getcwd()


# Path to the parsed list file

# Get all files in the downloads directory
downloads_dir = os.path.join(__dir__, "PDF")
#downloads_dir = os.path.join(__dir__, "PDF")
all_files = [os.path.join(downloads_dir, f) for f in os.listdir(downloads_dir) if f.endswith(".pdf")]

# Process files not in the parsed list
for pdf_file_name in all_files:
    #try:
    name_without_extension = '.'.join(os.path.basename(pdf_file_name).split('.')[0:-1])
    print(name_without_extension)

    # Prepare environment
    local_image_dir = os.path.join(__dir__, "PDFoutput", name_without_extension, "images")
    local_md_dir = os.path.join(__dir__, "PDFoutput", name_without_extension)
    image_dir = str(os.path.basename(local_image_dir))
    os.makedirs(local_image_dir, exist_ok=True)

    image_writer, md_writer = FileBasedDataWriter(local_image_dir), FileBasedDataWriter(local_md_dir)

    # Read bytes
    reader1 = FileBasedDataReader("")
    pdf_bytes = reader1.read(pdf_file_name)

    # Create Dataset Instance
    ds = PymuDocDataset(pdf_bytes)

    # Inference
    if ds.classify() == SupportedPdfParseMethod.OCR:
        infer_result = ds.apply(doc_analyze, ocr=True)
        print("OCRmode")
        pipe_result = infer_result.pipe_ocr_mode(image_writer)
    else:
        infer_result = ds.apply(doc_analyze, ocr=False) #class InferenceResult(InferenceResultBase):
        print("TXTmode")
        pipe_result = infer_result.pipe_txt_mode(image_writer)

    # Draw layout and spans results
    model_inference_result = infer_result.get_infer_res()
    md_content = pipe_result.get_markdown(image_dir)
    middle_json_content = pipe_result.get_middle_json()
    content_list_content = pipe_result.get_content_list(image_dir)
    print("Dumping")
    pipe_result.draw_layout(os.path.join(local_md_dir, f"{name_without_extension}_layout.pdf"))
    pipe_result.draw_span(os.path.join(local_md_dir, f"{name_without_extension}_spans.pdf"))
    pipe_result.dump_md(md_writer, f"{name_without_extension}.md", image_dir)
    pipe_result.dump_content_list(md_writer, f"{name_without_extension}_content_list.json", image_dir)
    pipe_result.dump_middle_json(md_writer, f'{name_without_extension}_middle.json')

        # Add the file to the parsed list

    #except Exception as e:
    #    print(f"Error processing file {pdf_file_name}: {e}")



2025-05-22 16:00:02.334 | INFO     | magic_pdf.data.dataset:__init__:157 - lang: None


2002.06187v1.Reusing_Static_Analysis_across_Different_Domain_Specific_Languages_using_Reference_Attribute_Grammars


2025-05-22 16:00:02.953 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 27342, cid_chars_radio: 0.0
2025-05-22 16:00:08.333 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:may_batch_image_analyze:275 - gpu_memory: 24 GB, batch_ratio: 16
2025-05-22 16:00:08.337 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:68 - DocAnalysis init, this may take some times, layout_model: doclayout_yolo, apply_formula: True, apply_ocr: True, apply_table: True, table_model: rapid_table, lang: None
2025-05-22 16:00:08.338 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:82 - using device: cuda
2025-05-22 16:00:08.338 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:86 - using models_dir: C:\Users\zhuzhanwen\.cache\huggingface\hub\models--opendatalab--PDF-Extract-Kit-1.0\snapshots\95817b4b2321769155f05c8d7e2f5a6b6da9e662/models
2025-05-22 16:00:21.167 | INFO     | magic_pdf.model.pdf_extract_kit:__init__:164 - DocAnalysis init done!
2025-05-22 

TXTmode


OCR-rec Predict: 100%|██████████| 2/2 [00:00<00:00, 108.06it/s]


Dumping


In [51]:
s="1610.00402v2.Dynamic_Polygon_Clouds__Representation_and_Compression_for_VR_AR.pdf"
s.split('.')[0:2]
result = '.'.join(s.split('.')[0:2])
result

'1610.00402v2'